# Multi-Modal AI Chatbot for Car Rental Company

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3
import base64
from io import BytesIO
from PIL import Image

/Users/huzaifa/projects/basic-multi-modal-chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
vehicle_id_to_name = {
    "Toyota Coaster": "1b764193-d727-4ad5-a5bf-9c6ffc6bd854",
    "Mercedes G-Wagon": "23bd3571-a8d1-4bda-92cc-e769a00fa9cc",
    "Toyota Hiace 2018": "2e106f4e-1f32-4f33-886b-2be7fa62379b",
    "Toyota Land Cruiser 2020": "3a515596-abef-4484-9076-e49e71dce378",
    "Mercedes Sprinter": "7031459c-fde3-4625-bf04-d15734f8d2da",
    "Lexus GX 460": "71f073e1-4c04-4330-acd5-740735386405",
    "Toyota Hilux": "7b944cba-9110-44ba-ad47-2581990d8ee1",
    "Lexus LX 470": "a52c194a-411b-4d4b-8147-9ed45309f6eb",
    "Toyota Camry": "acf8c50a-3fc4-4f63-9db3-f2bbc3eb07a5",
    "Toyota Prado": "b59a2c6a-6dc1-4cfd-bf03-9ab83a1e9130",
    "Hyundai H1": "b9e1cc17-9620-4b04-8fad-614a014138a2",
    "Toyota Land Cruiser": "f089f58c-aed3-44e1-b7fb-2ed7b4b41281",
    "Toyota Hiace": "fb1f6d9d-2121-4a4b-a0c6-c9919414103e"
}

vehicle_names = ["Toyota Coaster", "Mercedes G-Wagon", "Toyota Hiace 2018", "Toyota Land Cruiser 2020", "Mercedes Sprinter", "Lexus GX 460", "Toyota Hilux", "Lexus LX 470", "Toyota Camry", "Toyota Prado", "Hyundai H1", "Toyota Land Cruiser", "Toyota Hiace"]
service_types = ["airport_pickup", "hourly_rate", "full_day_8hr", "full_24hr", "border_states_roundtrip"]

In [4]:
# System message

system_message = f"""You are a helpful assistant for Exclusive Car Rentals. Their website is \
https://www.ecr.ng. Exclusive Car Rentals is a vehicle rental company. You are to \
help the user find the best vehicle for their needs and guide them through the booking process. The \
booking process is as follows: The user select the desired service, then select the vehicle, then \
enter their details. They will then get contacted by a customer service representative.
Always make sure the conversation is polite and friendly \
and stays within the scope of the website. If you do not know the answer, just say so.
Here is a list of available vehicles as-is in the DB:
{vehicle_names}
Use the vehicle name as-is when searching for a vehicle price, and 
The cost of renting a vehicle depends on the requested service type. Each vehicle has these 4 \
service types: {service_types}
Use a service type name as-is when searching using a service type. If a user doesn't specify a \
service type, search by airport_pickup.
"""

In [5]:
VEHICLE_DB = "vehicle.db"
VEHICLE_PRICES_DB = "vehicle_prices.db"

In [6]:
def get_vehicle_price(vehicle_name, service_type):
    vehicle_id = vehicle_id_to_name[vehicle_name]
    print(f"Checking the DB for vehicle ID {vehicle_id} and service type {service_type}", flush=True)
    with sqlite3.connect(VEHICLE_PRICES_DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM vehicle_prices WHERE vehicle_id = ? AND service_type = ?", (vehicle_id, service_type))
        result = cursor.fetchone()
        return f"The price for its {service_type} is {result[0]}" if result else "No price found"

In [7]:
vehicle_price_function = {
    "name": "get_vehicle_price",
    "description": "Get the rental price for a specific vehicle and service type. Use this when the customer asks about pricing for a vehicle rental.",
    "parameters": {
        "type": "object",
        "properties": {
            "vehicle_name": {
                "type": "string",
                "description": "The name of the vehicle the customer wants to rent (e.g., 'Toyota Coaster', 'Mercedes G-Wagon', 'Toyota Hiace'). Use the exact name from the available vehicles."
            },
            "service_type": {
                "type": "string",
                "description": "The type of rental service requested. Must be one of: 'hourly_rate' (hourly rental), 'full_day_8hr' (8-hour full day rental), 'full_24hr' (24-hour rental), 'airport_pickup' (airport pickup service), or 'border_states_roundtrip' (round trip to border states)."
            }
        },
        "required": ["vehicle_name", "service_type"],
        "additionalProperties": False
    }
}

In [8]:
tools = [{"type": "function", "function": vehicle_price_function}]

In [9]:
def vehicle_image(vehicle_name):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image of {vehicle_name}",
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )
    image_b64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_b64)
    return Image.open(BytesIO(image_data))

In [10]:
def audio(message):
    chat_audio = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=message
    )
    return chat_audio.content

In [11]:
def multi_modal_chat(history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    vehicle_name = None
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses, vehicle_name = handle_tool_calls_and_return_vehicle_name(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    voice = audio(reply)

    if vehicle_name:
        image = vehicle_image(vehicle_name)
    
    return history, voice, image

In [12]:
def handle_tool_calls_and_return_vehicle_name(message):
    responses = []
    vehicle_name = None
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_vehicle_price":
            arguments = json.loads(tool_call.function.arguments)
            vehicle_name = arguments.get("vehicle_name")
            service_type = arguments.get("service_type")
            price_info = get_vehicle_price(vehicle_name, service_type)
            responses.append({
                "role": "tool",
                "content": price_info,
                "tool_call_id": tool_call.id
            })
    return responses, vehicle_name

In [14]:
# Callbacks (along with the multi_modal_chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        multi_modal_chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


Checking the DB for vehicle ID b9e1cc17-9620-4b04-8fad-614a014138a2 and service type border_states_roundtrip
